Include march_crime.csv, Demograph.csv, no_dupes.csv and eve_new.csv
Take input.csv as training data for model

In [2]:
import numpy as np
import scipy as sp
import scipy.sparse
import pandas as pd
crimemarch = pd.read_csv("march_crime.csv")
demo = pd.read_csv("Demograph.csv")
d = demo[["NEIGHBORHO", "pop", "white"]]

beat_group = crimemarch.groupby(["neighborhood", "beat"], as_index=False).count()
beat_group = beat_group[["neighborhood", "beat", "offense_id"]]
beat_group = beat_group.sort_values(by=["beat", "offense_id"])
beat_group_2 = beat_group.groupby(["beat"]).max("offense_id")

beat_group = beat_group_2.merge(beat_group, on=["beat", "offense_id"], how="left")
n = []
for i in beat_group["neighborhood"].tolist():
  for j in d["NEIGHBORHO"]:
    if i in j:
      n.append(j)
      break
beat_group["NEIGHBORHO"] = n
beat_group = beat_group.merge(d, on="NEIGHBORHO", how="left")
beat_group.head()
model_data = pd.read_csv("no_dupes.csv")
model_data["beat"] = model_data["beat"].astype("str")
beat_group  = beat_group[["beat", "offense_id"]]
print(beat_group)
model_data = model_data.merge(beat_group, on="beat", how="left")
events = pd.read_csv("eve_new.csv")
print(events)
events_count = events.groupby("beat", as_index=False).count()

events_count["events_count"] = events_count["date"]
events_count["beat"] = events_count["beat"].astype(str)
model_data = model_data.merge(events_count, on="beat",  how="left")
model_data["true_crime"] = model_data["offense_id"]
model_data.to_csv("input.csv")


   beat  offense_id
0   101           6
1   102          18
2   103          22
3   104           8
4   105           7
..  ...         ...
74  610          11
75  611          12
76  612           9
77  613          11
78  614          11

[79 rows x 2 columns]
         date  beat
0    2/3/2021   605
1    3/3/2021   501
2    6/3/2021   301
3    6/3/2021   111
4    7/3/2021   104
5    8/3/2021   507
6   13/3/2021   209
7   14/3/2021   507
8   15/3/2021   507
9   16/3/2021   212
10  19/3/2021   507
11  20/3/2021   507
12  20/3/2021   303
13  23/3/2021   201
14  27/3/2021   511
15  28/3/2021   106
16  28/3/2021   209


Recompile predictions to include all beats.
Include source_data.csv, final_output.csv(model predictions)


In [3]:
source = pd.read_csv("source_data.csv")
final = pd.read_csv("final_output.csv")
source = source[["Beat", "NEIGHBORHO"]]
source = source.merge(final, on="NEIGHBORHO", how="left")
source.head(n=20)
source.to_csv("output_update.csv")
